In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv(r'C:\Users\tothm\PycharmProjects\Projet 7\datasets\application_test.csv')

data = df.copy()

In [3]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
with open('../model_weights/clf_xgb_o.pkl', 'rb') as f:
    model = joblib.load(f)

with open('../model_weights/imputer.pkl', 'rb') as f:
    imputer = joblib.load(f)

with open('../model_weights/colonnes_attendues.pkl', 'rb') as f:
    colonnes_model = joblib.load(f)

with open('../model_weights/scaler.pkl', 'rb') as f:
    scaler = joblib.load(f)

C:\Users\tothm\.virtualenvs\Projet_7-_BujKIZF\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.2.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tothm\.virtualenvs\Projet_7-_BujKIZF\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# Prépare la requête pour qu'elle soit conforme au modèle
# ONE HOT ENCODING
data = pd.get_dummies(data)
print("Étape 3 réussie.")

# VALEURS ABERRANTES
# Create an anomalous flag column
data['DAYS_EMPLOYED_ANOM'] = data["DAYS_EMPLOYED"] == 365243
# Replace the anomalous values with nan
data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
print("Étape 4 réussie.")

# Traitement des valeurs négatives
data['DAYS_BIRTH'] = abs(data['DAYS_BIRTH'])
print("Étape 5 réussie.")

# CREATION DE VARIABLES
# CREDIT_INCOME_PERCENT: the percentage of the credit amount relative to a client's income
# ANNUITY_INCOME_PERCENT: the percentage of the loan annuity relative to a client's income
# CREDIT_TERM: the length of the payment in months (since the annuity is the monthly amount due
# DAYS_EMPLOYED_PERCENT: the percentage of the days employed relative to the client's age

data['CREDIT_INCOME_PERCENT'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']
data['ANNUITY_INCOME_PERCENT'] = data['AMT_ANNUITY'] / data['AMT_INCOME_TOTAL']
data['CREDIT_TERM'] = data['AMT_ANNUITY'] / data['AMT_CREDIT']
data['DAYS_EMPLOYED_PERCENT'] = data['DAYS_EMPLOYED'] / data['DAYS_BIRTH']
print("Étape 6 réussie.")

# Récupère les colonnes attendues par le modèle
colonnes_attendues = colonnes_model
print(colonnes_attendues)
print("Étape 7 réussie.")

# Identify the columns that are missing from the received dataframe
missing_columns = set(colonnes_attendues) - set(data.columns)
print("Étape 8 réussie.")

# Add the missing columns to the received dataframe with a default value
for col in missing_columns:
    data[col] = 0
print("Étape 9 réussie.")

# Reorder the columns to match the order of the expected columns
data = data[colonnes_attendues]
data = data.reindex(columns=colonnes_attendues)
print("Étape 10 réussie.")

# Transform the data using the imputer and scaler
data = data[imputer.feature_names_in_]
data = imputer.transform(data)
print("Étape 11 réussie.")
data = scaler.transform(data)
print("Étape 12 réussie.")

# Make a prediction using the model
prediction = model.predict(data)
prediction_proba = model.predict_proba(data)
print("Étape 13 réussie.")

Étape 3 réussie.
Étape 4 réussie.
Étape 5 réussie.
Étape 6 réussie.
Index(['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       ...
       'WALLSMATERIAL_MODE_Panel', 'WALLSMATERIAL_MODE_Stone, brick',
       'WALLSMATERIAL_MODE_Wooden', 'EMERGENCYSTATE_MODE_No',
       'EMERGENCYSTATE_MODE_Yes', 'DAYS_EMPLOYED_ANOM',
       'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM',
       'DAYS_EMPLOYED_PERCENT'],
      dtype='object', length=243)
Étape 7 réussie.
Étape 8 réussie.
Étape 9 réussie.
Étape 10 réussie.
Étape 11 réussie.
Étape 12 réussie.
Étape 13 réussie.


In [6]:
prediction_proba

array([[0.81655085, 0.18344916],
       [0.5361401 , 0.46385992],
       [0.8010666 , 0.19893342],
       ...,
       [0.8415923 , 0.1584077 ],
       [0.869579  , 0.13042097],
       [0.36732113, 0.63267887]], dtype=float32)

In [7]:
results = pd.DataFrame(prediction_proba, columns=["Prediction : 0", "Prediction : 1"])

In [8]:
df = pd.concat([df, results], axis=1)

In [9]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Prediction : 0,Prediction : 1
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.816551,0.183449
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.536140,0.463860
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0.0,0.0,0.0,0.0,1.0,4.0,0.801067,0.198933
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.886541,0.113459
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.446496,0.553504


In [14]:
df.shape

(48744, 123)

In [11]:
df = df[df["CODE_GENDER"] != "XNA"]

In [12]:
df.to_csv("app_test_dashboard_with_prediction", index=False)

In [13]:
df.groupby(by="CODE_GENDER")["Prediction : 0", "Prediction : 1"].mean()

C:\Users\tothm\AppData\Local\Temp\ipykernel_13244\3172894126.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(by="CODE_GENDER")["Prediction : 0", "Prediction : 1"].mean()


,Prediction : 0,Prediction : 1
CODE_GENDER,,
F,0.638395,0.361605
M,0.503377,0.496623
